# GCP Skills Boost Updates

In [1]:
from bs4 import BeautifulSoup
import requests
import shutil
import multiprocessing
import pandas as pd
import datetime
import os
import math
import json

## Get html elements

In [2]:
url_base = 'https://www.cloudskillsboost.google'
url_catalog = f'{url_base}/catalog'
file_name = 'gcp_resources'

In [3]:
html = requests.get(url_catalog).text
soup = BeautifulSoup(html, 'html.parser')
num_resources = int(soup.find('p', attrs={'class': 'ql-body-2'}).string.split()[0])
num_pages = math.ceil(num_resources/10)

print(f'{num_resources} resources')

1021 resources


In [4]:
def get_urls():
    for i in range(num_pages):
        yield f'{url_catalog}?page={i+1}'
        
def get_format(element):
    return element.div.h3.a['data-type']

def get_name(element):
    return element.div.h3.a.text

def get_link(element):
    id = element.div.h3.a['href'].split('?')[0][1:]
    link = f'{url_base}/{id}'

    if id.startswith('focuses'):
        link += '?parent=catalog'

    return link

def get_description(element):
    return element.p.text.strip()

def get_level(element):
    tag = element.footer.find('div', 'catalog-item-level')
    level = tag.text.strip() if tag else ''

    return level

def get_credits(element):
    tag = element.footer.find('div', 'catalog-item-cost')
    tag_text = tag.text.strip()
    credits = int(tag_text.split()[0]) if tag_text != 'Free' else 0

    return credits

# status: development
def get_duration(element):
    duration = None
    duration_tag = element.footer.find('div', 'catalog-item-duration')
    if duration_tag:
        tokens = duration_tag.text.strip()
        if tokens == 4:
            hours = tokens[0]
            minutes = tokens[2]
        elif tokens == 2:
            duration_str, unit = tokens
            duration = int(duration_str)

            if unit == 'Hours':
                duration *= 60
            elif unit == 'day':
                duration *= 1440

In [5]:
def get_elements(url):
    html = requests.get(url).text
    soup = BeautifulSoup(html)
    elements = soup.find_all('li', 'catalog__list__item')

    for element in elements:
        data = {
            'format':       get_format(element),
            'name':         get_name(element),
            'link':         get_link(element),
            'description':  get_description(element),
            'credits':      get_credits(element),
            'level':        get_level(element)
        }

        with open(file_name, 'a') as f:
            f.write(json.dumps(data) + ',')

with multiprocessing.Pool(multiprocessing.cpu_count()) as p:
    p.map(get_elements, get_urls())

## Load data 

In [6]:
with open(file_name) as f:
    raw_data = f.read()

os.remove(file_name)

In [7]:
data = json.loads(f'[{raw_data[:-1]}]')
actual_data = pd.DataFrame(data)
print(f'{len(actual_data)} resources')

assert num_resources == len(actual_data)

1021 resources


In [8]:
last_data_file = '../data/gcp_resources_chkp_230624.json'
last_data = pd.read_json(last_data_file, orient='split')

## Preprocessing

In [9]:
actual_data[actual_data.duplicated()][['format', 'name']]

,format,name
243,Quest,Optimizing Your Google Cloud Costs
964,Course,Discovery AI


In [10]:
actual_data.drop_duplicates(inplace=True)

print(f'{len(actual_data)} resources')

1019 resources


In [11]:
actual_data = actual_data[~actual_data['name'].str.endswith('Locales')]
actual_data = actual_data.sort_values(by=['format','name', 'link'])

print(f'{len(actual_data)} resources')

948 resources


## Save actual data

In [12]:
timestamp = datetime.datetime.now().strftime('%y%m%d')

shutil.copy(last_data_file, f'{last_data_file}_tmp')
new_file_name = f'{file_name}_chkp_{timestamp}.json'
actual_data.to_json(f'../data/{new_file_name}', indent=4, orient='split', index=False)
print(f'{new_file_name} created!')

gcp_resources_chkp_230706.json created!


## Comparison

In [13]:
last_data['df'] = 'last'
actual_data['df'] = 'actual'

updates = pd.concat([last_data, actual_data])
updates['status'] = 'same'

same_names = updates.duplicated(keep=False, subset=['format', 'name'])
same_links = updates.duplicated(keep=False, subset=['link'])
last_df = updates['df'] == 'last'
actual_df = updates['df'] == 'actual'

updates.loc[last_df   & ~same_links, 'status'] = 'removed'
updates.loc[actual_df & ~same_links, 'status'] = 'new'
updates.loc[actual_df & ~same_links & same_names, 'status'] = 'new_same_name'
updates.loc[last_df   & same_links & ~same_names, 'status'] = 'old_name'
updates.loc[actual_df & same_links & ~same_names, 'status'] = 'new_name'

updates = updates[updates['status'] != 'same'].sort_values(by=['format', 'status', 'name'])

cols = ['format', 'name', 'status', 'link']

updates['link'] = updates['link'].apply(
    lambda x: f'<a href="{x}" target="_blank">link</a>')

updates[cols].style.format()

,format,name,status,link
782,Course,AppSheet Team Hierarchy,new,link
1016,Course,Introduction to Vertex Forecasting and Time Series in Practice,new_name,link
161,Course,Vertex Forecasting and Time Series in Practice,old_name,link
213,Lab,Assessing Data Quality with Dataplex,new,link
840,Lab,Build a Data Mesh with Dataplex: Challenge Lab,new,link
841,Lab,Cloud Speech API 3 Ways: Challenge Lab,new,link
676,Lab,Configure Service Accounts and IAM for Google Cloud: Challenge Lab,new,link
29,Lab,HTTPS Content-Based Load Balancer with Terraform,new,link
844,Lab,Implementing Security in Dataplex,new,link
503,Lab,Integrate BigQuery Data and Google Workspace using Apps Script: Challenge Lab,new,link


In [14]:
# Remove temporal Trivia resources
updates = updates[~updates['name'].str.contains('Trivia')]

In [15]:
# Copy markdown to clipboard. 
# Add to the Changelog.
pd.io.clipboards.to_clipboard(updates[cols].to_markdown(index=False), excel=False)
print(f'timestamp: {timestamp}')

timestamp: 230706


## Changelog

### 230706

| format   | name                                                                                    | status   | link                                                                                                |
|:---------|:----------------------------------------------------------------------------------------|:---------|:----------------------------------------------------------------------------------------------------|
| Course   | AppSheet Team Hierarchy                                                                 | new      | <a href="https://www.cloudskillsboost.google/course_templates/565" target="_blank">link</a>         |
| Course   | Introduction to Vertex Forecasting and Time Series in Practice                          | new_name | <a href="https://www.cloudskillsboost.google/course_templates/511" target="_blank">link</a>         |
| Course   | Vertex Forecasting and Time Series in Practice                                          | old_name | <a href="https://www.cloudskillsboost.google/course_templates/511" target="_blank">link</a>         |
| Lab      | Assessing Data Quality with Dataplex                                                    | new      | <a href="https://www.cloudskillsboost.google/focuses/67211?parent=catalog" target="_blank">link</a> |
| Lab      | Build a Data Mesh with Dataplex: Challenge Lab                                          | new      | <a href="https://www.cloudskillsboost.google/focuses/67212?parent=catalog" target="_blank">link</a> |
| Lab      | Cloud Speech API 3 Ways: Challenge Lab                                                  | new      | <a href="https://www.cloudskillsboost.google/focuses/67215?parent=catalog" target="_blank">link</a> |
| Lab      | Configure Service Accounts and IAM for Google Cloud: Challenge Lab                      | new      | <a href="https://www.cloudskillsboost.google/focuses/67219?parent=catalog" target="_blank">link</a> |
| Lab      | HTTPS Content-Based Load Balancer with Terraform                                        | new      | <a href="https://www.cloudskillsboost.google/focuses/1206?parent=catalog" target="_blank">link</a>  |
| Lab      | Implementing Security in Dataplex                                                       | new      | <a href="https://www.cloudskillsboost.google/focuses/67213?parent=catalog" target="_blank">link</a> |
| Lab      | Integrate BigQuery Data and Google Workspace using Apps Script: Challenge Lab           | new      | <a href="https://www.cloudskillsboost.google/focuses/67217?parent=catalog" target="_blank">link</a> |
| Lab      | Building an Application with MongoDB Atlas and Natural Language API hosted on Cloud Run | removed  | <a href="https://www.cloudskillsboost.google/focuses/50354?parent=catalog" target="_blank">link</a> |
| Quest    | Build a Data Mesh with Dataplex                                                         | new      | <a href="https://www.cloudskillsboost.google/quests/325" target="_blank">link</a>                   |
| Quest    | Cloud Speech API: 3 Ways                                                                | new      | <a href="https://www.cloudskillsboost.google/quests/326" target="_blank">link</a>                   |
| Quest    | Configure Service Accounts and IAM Roles for Google Cloud                               | new      | <a href="https://www.cloudskillsboost.google/quests/328" target="_blank">link</a>                   |
| Quest    | Integrate BigQuery Data and Google Workspace using Apps Script                          | new      | <a href="https://www.cloudskillsboost.google/quests/327" target="_blank">link</a>                   |
| Quest    | Measure Site Reliability using Cloud Operations Suite                                   | new      | <a href="https://www.cloudskillsboost.google/quests/189" target="_blank">link</a>                   |

### 230624

| format   | name                                                                      | status   | link                                                                                                |
|:---------|:--------------------------------------------------------------------------|:---------|:----------------------------------------------------------------------------------------------------|
| Lab      | Analyze Sentiment with Natural Language API: Challenge Lab                | new      | <a href="https://www.cloudskillsboost.google/focuses/66586?parent=catalog" target="_blank">link</a> |
| Lab      | Build and Deploy Machine Learning Solutions with Vertex AI: Challenge Lab | new      | <a href="https://www.cloudskillsboost.google/focuses/22019?parent=catalog" target="_blank">link</a> |
| Lab      | Develop with Apps Script and AppSheet: Challenge Lab                      | new      | <a href="https://www.cloudskillsboost.google/focuses/66584?parent=catalog" target="_blank">link</a> |
| Lab      | Google Workspace Admin: Managing Google Meet                              | new      | <a href="https://www.cloudskillsboost.google/focuses/18361?parent=catalog" target="_blank">link</a> |
| Lab      | Block.one: Getting Started with The EOSIO Blockchain                      | removed  | <a href="https://www.cloudskillsboost.google/focuses/18269?parent=catalog" target="_blank">link</a> |
| Lab      | Confluent: Developing a Streaming Microservices Application               | removed  | <a href="https://www.cloudskillsboost.google/focuses/13517?parent=catalog" target="_blank">link</a> |
| Quest    | Analyze Sentiment with Natural Language API                               | new      | <a href="https://www.cloudskillsboost.google/quests/323" target="_blank">link</a>                   |
| Quest    | Develop with Apps Script and AppSheet                                     | new      | <a href="https://www.cloudskillsboost.google/quests/324" target="_blank">link</a>                   |

### 230620

| format   | name                                                                      | status   | link                                                                                                |
|:---------|:--------------------------------------------------------------------------|:---------|:----------------------------------------------------------------------------------------------------|
| Course   | Developing Applications with Cloud Run on Google Cloud: Fundamentals      | new      | <a href="https://www.cloudskillsboost.google/course_templates/559" target="_blank">link</a>         |
| Course   | MySQL to Cloud Spanner                                                    | new      | <a href="https://www.cloudskillsboost.google/course_templates/560" target="_blank">link</a>         |
| Lab      | Enhancing Network Security Approach on Google Cloud                       | new      | <a href="https://www.cloudskillsboost.google/focuses/57194?parent=catalog" target="_blank">link</a> |
| Lab      | Managing Google Classroom                                                 | new      | <a href="https://www.cloudskillsboost.google/focuses/27154?parent=catalog" target="_blank">link</a> |
| Lab      | Networking Fundamentals on Google Cloud: Challenge Lab                    | new      | <a href="https://www.cloudskillsboost.google/focuses/65382?parent=catalog" target="_blank">link</a> |
| Lab      | The Basics of Google Cloud Compute: Challenge Lab                         | new      | <a href="https://www.cloudskillsboost.google/focuses/65384?parent=catalog" target="_blank">link</a> |
| Lab      | Use APIs to Work with Cloud Storage: Challenge Lab                        | new      | <a href="https://www.cloudskillsboost.google/focuses/65991?parent=catalog" target="_blank">link</a> |
| Lab      | Using the Google Cloud Speech API: Challenge Lab                          | new      | <a href="https://www.cloudskillsboost.google/focuses/65993?parent=catalog" target="_blank">link</a> |
| Lab      | Build and Deploy Machine Learning Solutions with Vertex AI: Challenge Lab | removed  | <a href="https://www.cloudskillsboost.google/focuses/22019?parent=catalog" target="_blank">link</a> |
| Lab      | Google Workspace Admin: Managing Google Meet                              | removed  | <a href="https://www.cloudskillsboost.google/focuses/18361?parent=catalog" target="_blank">link</a> |
| Lab      | Scaling VM-Series to Secure Google Cloud Networks                         | removed  | <a href="https://www.cloudskillsboost.google/focuses/57544?parent=catalog" target="_blank">link</a> |
| Lab      | Using Cloud Trace on Kubernetes Engine                                    | removed  | <a href="https://www.cloudskillsboost.google/focuses/5159?parent=catalog" target="_blank">link</a>  |
| Quest    | Networking Fundamentals on Google Cloud                                   | new      | <a href="https://www.cloudskillsboost.google/quests/318" target="_blank">link</a>                   |
| Quest    | The Basics of Google Cloud Compute                                        | new      | <a href="https://www.cloudskillsboost.google/quests/319" target="_blank">link</a>                   |
| Quest    | Use APIs to Work with Cloud Storage                                       | new      | <a href="https://www.cloudskillsboost.google/quests/321" target="_blank">link</a>                   |
| Quest    | Using the Google Cloud Speech API                                         | new      | <a href="https://www.cloudskillsboost.google/quests/320" target="_blank">link</a>                   |

### 230607

| format   | name                                                                    | status   | link                                                                                                |
|:---------|:------------------------------------------------------------------------|:---------|:----------------------------------------------------------------------------------------------------|
| Course   | Generative AI Fundamentals                                              | new      | <a href="https://www.cloudskillsboost.google/course_templates/556" target="_blank">link</a>         |
| Lab      | API Gateway: Qwik Start                                                 | new      | <a href="https://www.cloudskillsboost.google/focuses/17996?parent=catalog" target="_blank">link</a> |
| Lab      | Configure Your Workplace: Google Workspace for IT Admins: Challenge Lab | new      | <a href="https://www.cloudskillsboost.google/focuses/40783?parent=catalog" target="_blank">link</a> |
| Lab      | Google Workspace Admin: Getting Started                                 | new      | <a href="https://www.cloudskillsboost.google/focuses/16418?parent=catalog" target="_blank">link</a> |
| Lab      | Google Workspace Admin: Managing Applications                           | new      | <a href="https://www.cloudskillsboost.google/focuses/18332?parent=catalog" target="_blank">link</a> |
| Lab      | Google Workspace Admin: Managing Google Meet                            | new      | <a href="https://www.cloudskillsboost.google/focuses/18361?parent=catalog" target="_blank">link</a> |
| Lab      | Google Workspace Admin: Provisioning                                    | new      | <a href="https://www.cloudskillsboost.google/focuses/18119?parent=catalog" target="_blank">link</a> |
| Lab      | Google Workspace Admin: Securing                                        | new      | <a href="https://www.cloudskillsboost.google/focuses/18333?parent=catalog" target="_blank">link</a> |
| Lab      | Google Workspace Admin: Super Admin Account Recovery                    | new      | <a href="https://www.cloudskillsboost.google/focuses/20114?parent=catalog" target="_blank">link</a> |
| Lab      | Google Workspace for Education: Challenge Lab                           | new      | <a href="https://www.cloudskillsboost.google/focuses/29794?parent=catalog" target="_blank">link</a> |
| Lab      | Google Workspace for Education: Getting Started                         | new      | <a href="https://www.cloudskillsboost.google/focuses/25706?parent=catalog" target="_blank">link</a> |
| Lab      | Google Workspace for Education: Managing Services                       | new      | <a href="https://www.cloudskillsboost.google/focuses/28383?parent=catalog" target="_blank">link</a> |
| Lab      | Setting Up Google Meet for Distance Learning                            | new      | <a href="https://www.cloudskillsboost.google/focuses/27159?parent=catalog" target="_blank">link</a> |
| Lab      | Shared Drives: Getting Started                                          | new      | <a href="https://www.cloudskillsboost.google/focuses/5829?parent=catalog" target="_blank">link</a>  |
| Lab      | Teaching with Google Classroom                                          | new      | <a href="https://www.cloudskillsboost.google/focuses/25896?parent=catalog" target="_blank">link</a> |
| Lab      | HTTPS Content-Based Load Balancer with Terraform                        | removed  | <a href="https://www.cloudskillsboost.google/focuses/1206?parent=catalog" target="_blank">link</a>  |
| Quest    | Getting Started with Apache Beam                                        | new_name | <a href="https://www.cloudskillsboost.google/quests/310" target="_blank">link</a>                   |
| Quest    | Get Started with Apache Beam                                            | old_name | <a href="https://www.cloudskillsboost.google/quests/310" target="_blank">link</a>                   |

### 230604

| format   | name                                                                                                             | status   | link                                                                                                |
|:---------|:-----------------------------------------------------------------------------------------------------------------|:---------|:----------------------------------------------------------------------------------------------------|
| Course   | Introduction to Generative AI Studio                                                                             | new      | <a href="https://www.cloudskillsboost.google/course_templates/552" target="_blank">link</a>         |
| Course   | Introduction to Responsible AI                                                                                   | new      | <a href="https://www.cloudskillsboost.google/course_templates/554" target="_blank">link</a>         |
| Lab      | Analyze Images with the Cloud Vision API: Challenge Lab                                                          | new      | <a href="https://www.cloudskillsboost.google/focuses/64748?parent=catalog" target="_blank">link</a> |
| Lab      | Offloading Financial Mainframe Data into BigQuery and Elastic Search                                             | new      | <a href="https://www.cloudskillsboost.google/focuses/64744?parent=catalog" target="_blank">link</a> |
| Lab      | Protect Sensitive Data with Data Loss Prevention: Challenge Lab                                                  | new      | <a href="https://www.cloudskillsboost.google/focuses/64782?parent=catalog" target="_blank">link</a> |
| Lab      | Serverless Data Processing with Dataflow - Advanced Streaming Analytics Pipeline with Cloud Dataflow (Java)      | new      | <a href="https://www.cloudskillsboost.google/focuses/64777?parent=catalog" target="_blank">link</a> |
| Lab      | Serverless Data Processing with Dataflow - Testing with Apache Beam (Java)                                       | new      | <a href="https://www.cloudskillsboost.google/focuses/64778?parent=catalog" target="_blank">link</a> |
| Lab      | Serverless Data Processing with Dataflow - Writing an ETL Pipeline using Apache Beam and Cloud Dataflow (Python) | new      | <a href="https://www.cloudskillsboost.google/focuses/64780?parent=catalog" target="_blank">link</a> |
| Lab      | Serverless Data Processing with Dataflow - Writing an ETL pipeline using Apache Beam and Cloud Dataflow (Java)   | new      | <a href="https://www.cloudskillsboost.google/focuses/64779?parent=catalog" target="_blank">link</a> |
| Lab      | Install and Use Cloud Tools for PowerShell                                                                       | removed  | <a href="https://www.cloudskillsboost.google/focuses/666?parent=catalog" target="_blank">link</a>   |
| Lab      | VM Migration: Introduction to StratoZone Migrate                                                                 | removed  | <a href="https://www.cloudskillsboost.google/focuses/59254?parent=catalog" target="_blank">link</a> |
| Quest    | Analyze Images with the Cloud Vision API                                                                         | new      | <a href="https://www.cloudskillsboost.google/quests/308" target="_blank">link</a>                   |
| Quest    | Get Started with Apache Beam                                                                                     | new      | <a href="https://www.cloudskillsboost.google/quests/310" target="_blank">link</a>                   |
| Quest    | Protect Sensitive Data with Data Loss Prevention                                                                 | new      | <a href="https://www.cloudskillsboost.google/quests/315" target="_blank">link</a>                   |
| Quest    | Measure Site Reliability using Cloud Operations Suite                                                            | removed  | <a href="https://www.cloudskillsboost.google/quests/189" target="_blank">link</a>                   |

### 230528

| format   | name                                               | status        | link                                                                                                |
|:---------|:---------------------------------------------------|:--------------|:----------------------------------------------------------------------------------------------------|
| Course   | Media Rendering with Google Cloud                  | new           | <a href="https://www.cloudskillsboost.google/course_templates/549" target="_blank">link</a>         |
| Course   | Launching into Machine Learning                    | new_same_name | <a href="https://www.cloudskillsboost.google/course_templates/125" target="_blank">link</a>         |
| Lab      | Secure BigLake Data: Challenge Lab                 | new           | <a href="https://www.cloudskillsboost.google/focuses/64458?parent=catalog" target="_blank">link</a> |
| Lab      | Tag and Discover BigLake Data: Challenge Lab       | new           | <a href="https://www.cloudskillsboost.google/focuses/64167?parent=catalog" target="_blank">link</a> |
| Lab      | API Gateway: Qwik Start                            | removed       | <a href="https://www.cloudskillsboost.google/focuses/17996?parent=catalog" target="_blank">link</a> |
| Quest    | Getting Started with MongoDB Atlas on Google Cloud | new           | <a href="https://www.cloudskillsboost.google/quests/306" target="_blank">link</a>                   |
| Quest    | Secure BigLake Data                                | new           | <a href="https://www.cloudskillsboost.google/quests/307" target="_blank">link</a>                   |
| Quest    | Tag and Discover BigLake Data                      | new           | <a href="https://www.cloudskillsboost.google/quests/305" target="_blank">link</a>                   |

### 230522

| format   | name                                                                                    | status   | link                                                                                                |
|:---------|:----------------------------------------------------------------------------------------|:---------|:----------------------------------------------------------------------------------------------------|
| Lab      | Analyze Speech & Language with Google APIs: Challenge Lab                               | new      | <a href="https://www.cloudskillsboost.google/focuses/63851?parent=catalog" target="_blank">link</a> |
| Lab      | Create a Secure Data Lake on Cloud Storage: Challenge Lab                               | new      | <a href="https://www.cloudskillsboost.google/focuses/63857?parent=catalog" target="_blank">link</a> |
| Lab      | Monitoring in Google Cloud: Challenge Lab                                               | new      | <a href="https://www.cloudskillsboost.google/focuses/63855?parent=catalog" target="_blank">link</a> |
| Lab      | Building an Application with MongoDB Atlas and Natural Language API hosted on Cloud Run | new_name | <a href="https://www.cloudskillsboost.google/focuses/50354?parent=catalog" target="_blank">link</a> |
| Lab      | MongoDB Atlas with Natural Language API and Cloud Run                                   | old_name | <a href="https://www.cloudskillsboost.google/focuses/50354?parent=catalog" target="_blank">link</a> |
| Quest    | Analyze Speech and Language with Google APIs                                            | new      | <a href="https://www.cloudskillsboost.google/quests/302" target="_blank">link</a>                   |
| Quest    | Create a Secure Data Lake on Cloud Storage                                              | new      | <a href="https://www.cloudskillsboost.google/quests/303" target="_blank">link</a>                   |
| Quest    | Monitoring in Google Cloud                                                              | new      | <a href="https://www.cloudskillsboost.google/quests/304" target="_blank">link</a>                   |

### 230518

| format   | name                                                                 | status   | link                                                                                                |
|:---------|:---------------------------------------------------------------------|:---------|:----------------------------------------------------------------------------------------------------|
| Course   | Discovery AI                                                         | new_name | <a href="https://www.cloudskillsboost.google/course_templates/391" target="_blank">link</a>         |
| Course   | Product Discovery                                                    | old_name | <a href="https://www.cloudskillsboost.google/course_templates/391" target="_blank">link</a>         |
| Course   | Migrating to Google Cloud                                            | removed  | <a href="https://www.cloudskillsboost.google/course_templates/61" target="_blank">link</a>          |
| Lab      | Get Started with Generative AI Studio                                | new      | <a href="https://www.cloudskillsboost.google/focuses/63564?parent=catalog" target="_blank">link</a> |
| Lab      | Hosting a Web App on Google Cloud Using Compute Engine - Azure       | new      | <a href="https://www.cloudskillsboost.google/focuses/63541?parent=catalog" target="_blank">link</a> |
| Lab      | Enhancing Network Security Approach on Google Cloud                  | removed  | <a href="https://www.cloudskillsboost.google/focuses/57194?parent=catalog" target="_blank">link</a> |
| Lab      | Measure Site Reliability using Cloud Operations Suite: Challenge Lab | removed  | <a href="https://www.cloudskillsboost.google/focuses/27165?parent=catalog" target="_blank">link</a> |

### 230517

| format   | name                                                      | status   | link                                                                                                |
|:---------|:----------------------------------------------------------|:---------|:----------------------------------------------------------------------------------------------------|
| Course   | Create Image Captioning Models                            | new      | <a href="https://www.cloudskillsboost.google/course_templates/542" target="_blank">link</a>         |
| Course   | Encoder-Decoder Architecture                              | new      | <a href="https://www.cloudskillsboost.google/course_templates/543" target="_blank">link</a>         |
| Course   | Introduction to Image Generation                          | new      | <a href="https://www.cloudskillsboost.google/course_templates/541" target="_blank">link</a>         |
| Course   | Preparing for Your Google Workspace Administrator Journey | new      | <a href="https://www.cloudskillsboost.google/course_templates/540" target="_blank">link</a>         |
| Lab      | Qwiklabs Trivia May Week 3                                | new      | <a href="https://www.cloudskillsboost.google/focuses/63283?parent=catalog" target="_blank">link</a> |
| Lab      | SAP Landing Zone: Add and Configure Storage to SAP VMs    | new      | <a href="https://www.cloudskillsboost.google/focuses/63280?parent=catalog" target="_blank">link</a> |
| Lab      | SAP Landing Zone: Plan and Deploy SAP Virtual Machines    | new      | <a href="https://www.cloudskillsboost.google/focuses/63281?parent=catalog" target="_blank">link</a> |
| Lab      | SAP Landing Zone: Plan and Deploy the SAP Network         | new      | <a href="https://www.cloudskillsboost.google/focuses/63282?parent=catalog" target="_blank">link</a> |
| Lab      | Qwiklabs Trivia May Week 2                                | removed  | <a href="https://www.cloudskillsboost.google/focuses/62947?parent=catalog" target="_blank">link</a> |
| Quest    | Build an SAP Landing Zone on Google Cloud                 | new      | <a href="https://www.cloudskillsboost.google/quests/223" target="_blank">link</a>                   |
| Quest    | Google Cloud's Operations Suite on GKE                    | removed  | <a href="https://www.cloudskillsboost.google/quests/133" target="_blank">link</a>                   |

### 230515

| format   | name                                          | status   | link                                                                                                |
|:---------|:----------------------------------------------|:---------|:----------------------------------------------------------------------------------------------------|
| Lab      | Generative AI with Vertex AI: Getting Started | new      | <a href="https://www.cloudskillsboost.google/focuses/63250?parent=catalog" target="_blank">link</a> |
| Lab      | Generative AI with Vertex AI: Prompt Design   | new      | <a href="https://www.cloudskillsboost.google/focuses/63251?parent=catalog" target="_blank">link</a> |
| Quest    | Generative AI Explorer - Vertex AI            | new      | <a href="https://www.cloudskillsboost.google/quests/299" target="_blank">link</a>                   |

### 230513

| format   | name                                     | status   | link                                                                                                |
|:---------|:-----------------------------------------|:---------|:----------------------------------------------------------------------------------------------------|
| Lab      | App Engine: 3 Ways: Challenge Lab        | new      | <a href="https://www.cloudskillsboost.google/focuses/63241?parent=catalog" target="_blank">link</a> |
| Lab      | Get Started with Eventarc: Challenge Lab | new      | <a href="https://www.cloudskillsboost.google/focuses/63244?parent=catalog" target="_blank">link</a> |
| Lab      | Get Started with Pub/Sub: Challenge Lab  | new      | <a href="https://www.cloudskillsboost.google/focuses/63246?parent=catalog" target="_blank">link</a> |
| Quest    | App Engine: 3 Ways                       | new      | <a href="https://www.cloudskillsboost.google/quests/285" target="_blank">link</a>                   |
| Quest    | Get Started with Eventarc                | new      | <a href="https://www.cloudskillsboost.google/quests/300" target="_blank">link</a>                   |
| Quest    | Get Started with Pub/Sub                 | new      | <a href="https://www.cloudskillsboost.google/quests/301" target="_blank">link</a>                   |

### 230324 - 230510

| format | name                                                         | status   | link                                                                                        |
| :----- | :----------------------------------------------------------- | :------- | :------------------------------------------------------------------------------------------ |
| Course | Attention Mechanism                                          | new      | <a href="https://www.cloudskillsboost.google/course_templates/537" target="_blank">link</a> |
| Course | Building Applications with Eventarc on Google Cloud          | new      | <a href="https://www.cloudskillsboost.google/course_templates/531" target="_blank">link</a> |
| Course | Design Foundations for Streaming with Google Cloud           | new      | <a href="https://www.cloudskillsboost.google/course_templates/532" target="_blank">link</a> |
| Course | Developing Containerized Applications on Google Cloud        | new      | <a href="https://www.cloudskillsboost.google/course_templates/530" target="_blank">link</a> |
| Course | Introduction to Generative AI                                | new      | <a href="https://www.cloudskillsboost.google/course_templates/536" target="_blank">link</a> |
| Course | Introduction to Large Language Models                        | new      | <a href="https://www.cloudskillsboost.google/course_templates/539" target="_blank">link</a> |
| Course | Serving Multimedia Content with Google Cloud                 | new      | <a href="https://www.cloudskillsboost.google/course_templates/535" target="_blank">link</a> |
| Course | Transformer Models and BERT Model                            | new      | <a href="https://www.cloudskillsboost.google/course_templates/538" target="_blank">link</a> |
| Course | Deploy and Monitor in Google Cloud for AWS Professionals     | new_name | <a href="https://www.cloudskillsboost.google/course_templates/523" target="_blank">link</a> |
| Course | Deploy and Monitor in Google Cloud for Azure Professionals   | new_name | <a href="https://www.cloudskillsboost.google/course_templates/524" target="_blank">link</a> |
| Course | Google Cloud Compute and Scalability for AWS Professionals   | new_name | <a href="https://www.cloudskillsboost.google/course_templates/526" target="_blank">link</a> |
| Course | Google Cloud Compute and Scalability for Azure Professionals | new_name | <a href="https://www.cloudskillsboost.google/course_templates/525" target="_blank">link</a> |
| Course | Google Cloud IAM and Networking for AWS Professionals        | new_name | <a href="https://www.cloudskillsboost.google/course_templates/522" target="_blank">link</a> |
| Course | Google Cloud IAM and Networking for Azure Professionals      | new_name | <a href="https://www.cloudskillsboost.google/course_templates/520" target="_blank">link</a> |
| Course | Google Cloud Storage and Containers for AWS Professionals    | new_name | <a href="https://www.cloudskillsboost.google/course_templates/519" target="_blank">link</a> |
| Course | Google Cloud Storage and Containers for Azure Professionals  | new_name | <a href="https://www.cloudskillsboost.google/course_templates/521" target="_blank">link</a> |
| Course | Machine Learning Operations (MLOps): Getting Started         | new_name | <a href="https://www.cloudskillsboost.google/course_templates/158" target="_blank">link</a> |
| Course | MLOps (Machine Learning Operations) Fundamentals             | old_name | <a href="https://www.cloudskillsboost.google/course_templates/158" target="_blank">link</a> |
